In [30]:
#Trims Fastq files of polyA tails for Bowtie Output

file1 = open('D:/ZidLab/SequenceReads/Kyle_7_18_17/WT_5D_FP_S3_L003_R1_001.fastq' , 'r')
output = open('D:/Bowtie/bowtie-1.1.2/WT_5DFP_7_17.fastq' , 'w')
count = 0
count1 = 0
count2 = 0
num = 0
read = 0
for line in file1 :
        num+=1
        if num ==2:  #The file has sequences on the 2nd line, every 4 lines
                temp2 = line.rstrip()
                temp26 = temp2[0:25] #Take the first 26 bps
                tempminus1 = temp26.rstrip('A') #Strip all the A's
                tempminus2 = tempminus1.rstrip('N') #Strip the next base (in case an A was misread
                tempminusA = tempminus2.rstrip('A') #Strip more A's
                Length = len(tempminusA)
                tempTest = tempminusA
                tempTest1 = tempTest[0:-1]
                tempTest2 = tempTest1.rstrip('A')
                LenChange = Length - len(tempTest2) #I'm double checking for extra runs of A's, not sure if its needed.
                if LenChange > 6 :
                        count2 = count2 + 1
                        read = 7
                else:
                        if Length > 17 :
                                if Length > 23:
                                        Sequence = tempminusA[0:23] #Only use the first 23bps for alignment
                                        Length = len(Sequence)      #More likely to have misreads later in the sequence
                                else:
                                        Sequence = tempminusA
                                acount = Sequence.count('A')
                                temp4 = Length - 5
                                if acount < temp4 :
                                        count = count + 1
                                        read = 0
                                else:
                                        read = 7
                        else:
                                read = 7
                
        
        elif num == 4: #This is to get the Quality of the sequence read
                count1+=1
                num = 0
                QualityT = line.rstrip()
                Quality = QualityT[0:Length]
                if read == 7:
                        read = 0
                else:
                        output.write('@r'+str(count)+'/1.'+str(Length)+'\n'+Sequence+'\n'+'+\n'+Quality+'\n')
                
output.close()
print count
print count1
print count2
#Takes txt Solexa Sequence results and outputs them in fastq for
# Took the first 26bp, removed the 3'AAAAs

21475045
28030106
593662


In [5]:
#Converts Bowtie Output to counts per chromosome site
numerals = [
        {'letter': 'M', 'value': 1000},
        {'letter': 'D', 'value': 500},
        {'letter': 'C', 'value': 100},
        {'letter': 'L', 'value': 50},
        {'letter': 'X', 'value': 10},
        {'letter': 'V', 'value': 5},
        {'letter': 'I', 'value': 1},
    ]
def roman_to_arabic(number):
    index_by_letter = {}
    for index in xrange(len(numerals)):
        index_by_letter[numerals[index]['letter']] = index

    result = 0
    previous_value = None
    for letter in reversed(number):
        index = index_by_letter[letter]
        value = numerals[index]['value']
        if (previous_value is None) or (previous_value <= value):
            result += value
        else:
            result -= value
        previous_value = value

    return result



file1 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/unconwt135align.map','r') 
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wt135_AlignSortCount.txt','w')
count2 = 0
count3 =0
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
Chr = temp1[2]
#if ChrTemp2 == "MT":
#    Chr = 17
#else:
#    Chr = roman_to_arabic(ChrTemp2)

Name = temp1[0]
NameLen = len(temp1[3])
Orien = temp1[1]
Posit = (int(temp1[3]))
#if Orien == '-':
#    Posit = (int(temp1[3]))+(int(NameLen))  #didn't subtract 1 adjusting because it is one off
#else :
#    Posit = (int(temp1[3]))+1 #adjusting because it seems to be one off on the +Strand
    

dct_lst = [{"Chr":Chr,"Posit":Posit,"Orien": Orien}]#"Seq": 'a'}]

for line in file1 :
    count3 +=1
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    Chr = temp1[2]
    #if ChrTemp2 == "MT":
    #    Chr = 17
    #else:
    #    Chr = roman_to_arabic(ChrTemp2)
    Name = temp1[0]
    NameLen = len(temp1[3])
    Orien = temp1[1]
    Posit = (int(temp1[3]))
    #if Orien == '-':
    #    Posit = (int(temp1[3]))+(int(NameLen))
    #elif Orien =='+' :
    #    Posit = (int(temp1[3]))+1
    temp_lst = [{"Chr":Chr,"Posit":Posit,"Orien": Orien}]#"Seq": 'a'}]#temp1[4]
    dct_lst.extend(temp_lst)


from operator import itemgetter
dct_lst.sort(key=itemgetter("Posit"))

from operator import itemgetter
dct_lst.sort(key=itemgetter("Chr"))


count = 1
Length = len(dct_lst)
for i in range(len(dct_lst)) :
    Chr = (dct_lst[i]["Chr"])
    Posit = (dct_lst[i]["Posit"])
    Orien = (dct_lst[i]["Orien"])
    #Seq = (dct_lst[i]["Seq"])
    if i == (Length-1):
        output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n')
    else:
        OrienT2 = (dct_lst[i+1]["Orien"])
        PositT2 = (dct_lst[i+1]["Posit"])
        if Orien == OrienT2 :
        
            if Posit == PositT2 :
                count = count + 1
            else :
                count2 = count + count2
                output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n') #sense count
                count = 1
        else :
            count2 = count + count2
            output.write(Orien+'\t'+str(Chr)+'\t'+str(count)+'\t'+str(Posit)+'\t'+'\n')#Seq+'\n') #sense count
            count = 1
         
file1.close()
output.close()  
print (count2)

16673067


In [22]:
#Converts accumulated sorted data to Mochiview format
file1 = open('D:/ZidLab/SolexaDataAnalysis/WilliamsMito/880_AlignSortCount.txt','r')
output = open('D:/ZidData/OSheaLab/MochiView/5Pp35_comb_Mochi.txt', 'w')
output.write('SEQ_NAME\tSTART\tEND\tSTRAND\tCOUNT\n')
#temp7 = file1.readline()
norm = 2.65 # Normalized to total alignable reads/3 million

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    name = "Chr"+temp1[1]
       
    PositTemp = int(temp1[3])
    Orien = temp1[0]
    if Orien == '+' :
        count = (float(temp1[2]))/norm
        Start = str(PositTemp)
        
    else :
        tempc = (float(temp1[2]))/norm
        count = tempc*(-1)
        Start = str(PositTemp)
        
    output.write(name+'\t'+Start+'\t'+Start+'\t'+Orien+'\t'+str(count)+ '\n')  #antisense count


output.close()
file1.close()

In [7]:
#Converts accumulated sorted data at the genome level to gene features

file1 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aD137_AlignSortCount.txt','r')
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aD137_Feat.txt', 'w')
AlignCountTotal = 0.0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    
    Chr = temp1[1]
    Posit = int(temp1[3])
    Posit = Posit + 1
    Orien = temp1[0]
    count = temp1[2]
    AlignCount = 0
    
    #Seq = temp1[5]
    if Chr == 'Chr1' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND1.txt','r')
    elif Chr == 'Chr2' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND2.txt','r')
    elif Chr == 'Chr3' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND3.txt','r')
    elif Chr == 'Chr4' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND4.txt','r')
    elif Chr == 'Chr5' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND5.txt','r')
    elif Chr == 'Chr6' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND6.txt','r')
    elif Chr == 'Chr7' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND7.txt','r')
    elif Chr == 'Chr8' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND8.txt','r')
    elif Chr == 'Chr9' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND9.txt','r')
    elif Chr == 'Chr10' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND10.txt','r')
    elif Chr == 'Chr11' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND11.txt','r')
    elif Chr == 'Chr12' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND12.txt','r')
    elif Chr == 'Chr13' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND13.txt','r')
    elif Chr == 'Chr14' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND14.txt','r')
    elif Chr == 'Chr15' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND15.txt','r')
    elif Chr == 'Chr16' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND16.txt','r')
    elif Chr == 'Chr17' :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND17.txt','r')    
    
    for line2 in file2 :
        temp3 = line2.rstrip()
        temp2 = temp3.split('\t')
        Chr2 = 'Chr'+(temp2[1])
        if Chr == Chr2: #checking if the same chromosome
            if Orien == '-' :  # C orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '-' :  #also C orientation
                    tStrt2 = int(temp2[3])
                    tEnd2 = int(temp2[2])
                    Strt2 = tStrt2+16
                    End2 = tEnd2+13
                                    
                    if End2 < Posit < Strt2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+'\t'+count+'\n') #sense count
                
                  
            else :          # W orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '+' :  #also W orientation
                    tStrt2 = int(temp2[2])
                    tEnd2 = int(temp2[3])
                    Strt2 = tStrt2-16
                    End2 = tEnd2-13

                    if Strt2 < Posit < End2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+ '\t'+count+'\n') #sense count
                        AlignCountTotal = AlignCountTotal + float(count)
    file2.close()
print (AlignCountTotal)
print (Chr)
output.close()

8928965.0
Chr9


In [14]:
#Outputs data to excel file with counts per gene
import sys

file1 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aDcomb_Feat.txt','r')
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aDcomb_Sorted.txt','w')
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
Name = temp1[0]
ChrT = Name[2]
Chr = '1'
dct_lst = [{"name":temp1[0],"Start":temp1[1],"End":"7236","Posit":temp1[3],"Count":temp1[4]}]

for line in file1 :
   temp = line.rstrip()
   temp1 = temp.split('\t')
     
   temp_lst = [{"name":temp1[0],"Start":temp1[1],"End":temp1[2],"Posit":temp1[3],"Count":temp1[4]}]
   dct_lst.extend(temp_lst)
file1.close()

#for line in file2:
  
#   temp = line.rstrip()
#   temp1 = temp.split('\t')  
#   temp_lst = [{"name":temp1[0],"Start":temp1[1],"End":temp1[2],"Posit":temp1[3],"Count":temp1[4],"ASCount": temp1[5]}]
#   dct_lst.extend(temp_lst)

#file2.close()


from operator import itemgetter
dct_lst.sort(key=itemgetter("name"))


for i in range(len(dct_lst)) :
   output.write(dct_lst[i]["name"]+'\t'+dct_lst[i]["Start"]+'\t'+dct_lst[i]["End"]+'\t'+dct_lst[i]["Posit"]+'\t'+dct_lst[i]["Count"]+'\n') #sense count
      
output.close()

file1 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aDcomb_Sorted.txt')
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/eIF5aDcomb_Counted.txt','w')
output.write('Name\tStrt\tEnd\tLength\tCount\tStallCount\tCodeCount\t200Count\t200EndCount\n')


tName = 'YAL001C'
tStrt = 147596
tEnd = 151168
tLength = 3573
tCount = 0
Count =0
SCount = 0
CCount = 0
CountS = 0
CountE = 0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t')
    Name = temp1[0]
   
    if Name[0] == 'Q':
        Count = 0
    elif Name[0] == 'Y':
        Strt = int(temp1[1])
        End = int(temp1[2])
        Orien = Name[6]
        Length = abs(Strt-End)
        Posit = int(temp1[3])
        tCount = int(temp1[4])
        if tName == Name:
            Count = Count + tCount
            if Orien == 'W':
                if Strt+20 > Posit and Strt-16 < Posit:
                    SCount = SCount + tCount
                elif Strt+220 > Posit and Strt-16 < Posit:
                    CountS = CountS + tCount
                elif End - 200 < Posit:
                    CountE = CountE + tCount

            elif Orien == 'C':
                if Strt-20 < Posit and Strt+16 > Posit:
                    SCount = SCount + tCount
                elif Strt-220 < Posit and Strt+16 > Posit:
                    CountS = CountS + tCount
                elif End + 200 > Posit:
                    CountE = CountE + tCount

        else :
            CCount = Count - SCount
            output.write(tName+'\t'+str(tStrt)+'\t'+str(tEnd)+'\t'+str(tLength)+'\t'+str(Count)+'\t'+str(SCount)+'\t'+str(CCount)+'\t'+str(CountS)+'\t'+str(CountE)+'\n')
            Count =0
            SCount = 0
            CCount = 0
            CountS = 0
            CountE = 0
            tName = Name
            Count = Count + tCount
            tStrt = Strt
            tEnd = End
            tLength = Length
            if Orien == 'W':
                if Strt+20 > Posit and Strt-16 < Posit:
                    SCount = SCount + tCount
                elif Strt+220 > Posit and Strt-16 < Posit:
                    CountS = CountS + tCount
                elif End - 200 < Posit:
                    CountE = CountE + tCount

            elif Orien == 'C':
                if Strt-20 < Posit and Strt+16 > Posit:
                    SCount = SCount + tCount
                elif Strt-220 < Posit and Strt+16 > Posit:
                    CountS = CountS + tCount
                elif End + 200 > Posit:
                    CountE = CountE + tCount
output.close()   

In [26]:
#Converts accumulated sorted data at the genome level to gene features
##same as the ZidCode.py

file1 = open('C:/Zid/SolexaDataAnalysis/Anna_7_18_17/RNA3AlignSortCount.txt','r')
output = open('C:/Users/Brian/OneDrive/ZidLab/Data/Solexa/RNA3Feat.txt', 'w')
AlignCountTotal = 0.0

for line in file1:
    temp = line.rstrip()
    temp1 = temp.split('\t') 
    Chr = int(temp1[1])
    Posit = int(temp1[3])
    Posit = Posit + 1
    Orien = temp1[0]
    count = temp1[2]
    AlignCount = 0
    
    #Seq = temp1[5]
    if Chr == 1 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND1.txt','r')
    elif Chr == 2 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND2.txt','r')
    elif Chr == 3 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND3.txt','r')
    elif Chr == 4 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND4.txt','r')
    elif Chr == 5 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND5.txt','r')
    elif Chr == 6 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND6.txt','r')
    elif Chr == 7 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND7.txt','r')
    elif Chr == 8 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND8.txt','r')
    elif Chr == 9 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND9.txt','r')
    elif Chr == 10 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND10.txt','r')
    elif Chr == 11 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND11.txt','r')
    elif Chr == 12 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND12.txt','r')
    elif Chr == 13 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND13.txt','r')
    elif Chr == 14 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND14.txt','r')
    elif Chr == 15 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND15.txt','r')
    elif Chr == 16 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND16.txt','r')
    elif Chr == 17 :
        file2 = open('C:/Zid/SolexaDataAnalysis/Chrom/ORF_ND17.txt','r')    
    
    for line2 in file2 :
        temp3 = line2.rstrip()
        temp2 = temp3.split('\t')
        Chr2 = int(temp2[1])
        if Chr == Chr2: #checking if the same chromosome
            if Orien == '-' :  # C orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '-' :  #also C orientation
                    tStrt2 = int(temp2[3])
                    tEnd2 = int(temp2[2])
                    Strt2 = tStrt2+16
                    End2 = tEnd2+13
                                    
                    if End2 < Posit < Strt2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+'\t'+count+'\n') #sense count
                
                  
            else :          # W orientation
                name = temp2[0]
                Orien2 = temp2[4]
                if Orien2 == '+' :  #also W orientation
                    tStrt2 = int(temp2[2])
                    tEnd2 = int(temp2[3])
                    Strt2 = tStrt2-16
                    End2 = tEnd2-13

                    if Strt2 < Posit < End2 :
                        AlignCount = AlignCount + 1
                        output.write(name+'\t'+str(tStrt2)+'\t'+str(tEnd2)+'\t'+str(Posit)+ '\t'+count+'\n') #sense count
                        AlignCountTotal = AlignCountTotal + float(count)
    file2.close()
print AlignCountTotal
output.close()

820364.0


In [27]:
import pandas as pd
df = pd.read_csv('C:/Users/Brian/OneDrive/ZidLab/Data/Solexa/GenesRibo.txt',sep='\t',header=(0))

df2 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260106_WT_ribo_rep1_counts.txt',sep='\t',header=(0))
df3 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260107_WT_ribo_rep2_counts.txt',sep='\t',header=(0))
df4 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260108_WT_ribo_rep3_counts.txt',sep='\t',header=(0))
df5 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260109_can1_ribo_rep1_counts.txt',sep='\t',header=(0)) 
df6 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260110_can1_ribo_rep2_counts.txt',sep='\t',header=(0))
df7 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260111_can1_ribo_rep3_counts.txt',sep='\t',header=(0))
df8 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260112_WT_mRNA_rep1_counts.txt',sep='\t',header=(0))
df9 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260113_WT_mRNA_rep2_counts.txt',sep='\t',header=(0)) 
df10 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260114_can1_mRNA_rep1_counts.txt',sep='\t',header=(0))
df11 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260115_can1_mRNA_rep2_counts.txt',sep='\t',header=(0))
df12 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260116_rpl22a_ribo_rep1_counts.txt',sep='\t',header=(0))
df13 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260117_rpl22a_ribo_rep2_counts.txt',sep='\t',header=(0)) 
df14 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260118_tor1_ribo_rep1_counts.txt',sep='\t',header=(0))
df15 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260119_tor1_ribo_rep2_counts.txt',sep='\t',header=(0))
df16 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260120_rpl22a_mRNA_counts.txt',sep='\t',header=(0))
df17 = pd.read_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/GSE85198_RAW/GSM2260121_tor1_mRNA_counts.txt',sep='\t',header=(0)) 
new_df = df.merge(df2, how='left', on='Name').merge(df3, how='left', on='Name').merge(df4, how='left', on='Name').merge(df5, how='left', on='Name').merge(df6, how='left', on='Name').merge(df7, how='left', on='Name').merge(df8, how='left', on='Name').merge(df9, how='left', on='Name').merge(df10, how='left', on='Name').merge(df11, how='left', on='Name').merge(df12, how='left', on='Name').merge(df13, how='left', on='Name').merge(df14, how='left', on='Name').merge(df15, how='left', on='Name').merge(df16, how='left', on='Name').merge(df17, how='left', on='Name')

new_df.to_csv('D:/ZidLab/Solexa/OtherPeoples/YeastRplRibo/dataMerge.txt', sep='\t')


In [11]:
#Combine Feat Files
#Combine AlignCountSorts  10_8_2014
##ombine the counts if there is 2 gene with same name and same position.
file1 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wt134_Feat.txt','r')
file2 = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wt135_Feat.txt','r')

output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wtcomb_Feat.txt','w')

count = 0
temp = file1.readline()
line1 = temp.rstrip() 
temp1 = line1.split('\t')
dct_lst = [{"Name":temp1[0],"Start":(temp1[1]),"End":(temp1[2]),"Posit": temp1[3],"Count":int(temp1[4]),}]
for line in file1:
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    temp_lst = [{"Name":temp1[0],"Start":(temp1[1]),"End":(temp1[2]),"Posit": temp1[3],"Count":int(temp1[4]),}]
    dct_lst.extend(temp_lst)
for line in file2:
    line1 = line.rstrip() 
    temp1 = line1.split('\t')
    temp_lst = [{"Name":temp1[0],"Start":(temp1[1]),"End":(temp1[2]),"Posit": temp1[3],"Count":int(temp1[4]),}]
    dct_lst.extend(temp_lst)
    
from operator import itemgetter
dct_lst.sort(key=itemgetter("Posit"))

from operator import itemgetter
dct_lst.sort(key=itemgetter("Name"))

count2 = 0
CountT = 0
test = 0
Length = len(dct_lst)
for i in range(len(dct_lst)) :
    Name = (dct_lst[i]["Name"])
    Posit = (dct_lst[i]["Posit"])
    Start = (dct_lst[i]["Start"])
    End = (dct_lst[i]["End"])
    CountTemp = int(dct_lst[i]["Count"])
    
    if i == (Length-1):
        if test == 1:
            output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountT)+'\n')#Seq+'\n')
            count2 = CountT + count2
        else:
            output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountTemp)+'\n')
            count2 = CountTemp + count2
    else:
        NameT2 = (dct_lst[i+1]["Name"])
        StartT2 = (dct_lst[i+1]["Start"])
        EndT2 = (dct_lst[i+1]["End"])
        CountT2 = int(dct_lst[i+1]["Count"])
        PositT2 = (dct_lst[i+1]["Posit"])
        if Name == NameT2 :
        
            if Posit == PositT2 :
                test = 1
                CountT = CountT2 + CountTemp
            else :
                if test == 1:
                    output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountT)+'\n')#Seq+'\n')
                    count2 = CountT + count2
                    CountT = 0
                else:
                    output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountTemp)+'\t'+'\n')
                    count2 = CountTemp + count2
                    CountT = 0
                
                test = 0
        else :
            
            if test == 1:
                output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountT)+'\n')#Seq+'\n')
                count2 = CountT + count2
                CountT = 0
            else:
                output.write(Name+'\t'+Start+'\t'+End+'\t'+Posit+'\t'+str(CountTemp)+'\n')
                count2 = CountTemp + count2
                CountT = 0
            test = 0
         
output.close()  
print (count2)

36300355


In [46]:
#Trims Fastq files of polyA tails for Bowtie Output

file1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF_TTAGGC.fastq' , 'r')
output1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF-G_15m_TCTAG.fastq' , 'w')
output2 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RFP-G_5m_GCATA.fastq' , 'w')
output3 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RFP-G_1m_CTAGA.fastq' , 'w')
output4 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RFP+G_AGCTA.fastq' , 'w')
output5 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF-G_15m+Cyclo_ATCGT.fastq' , 'w')
output6 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RNA-G_1m_CGTAA.fastq' , 'w')
output7 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RNA-G_5m_GATCA.fastq' , 'w')
output8 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RNA-G_15m_TAGAC.fastq' , 'w')
output9 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF_noIndex.fastq' , 'w')

def find_str(s,char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index
                
            index +=1
        
    return -1

count = 0
count1 = 0
count2 = 0
num = 0
read = 0
sequence = "AGATCGGAAG"
exit = 0

for line in file1 :
    exit+=1
    #if exit == 10000:
       #break
    count +=1
    num+=1
    if num == 1:
        name1 =line.rstrip()
            
    elif num ==2:  #The file has sequences on the 2nd line, every 4 lines
        temp2 = str(line.rstrip())
                   
        index_site = (find_str(temp2, sequence))
        
        if index_site == -1:
            y=3
        else:
            indx = (temp2[index_site-5:index_site])
            umi = temp2[index_site-10:index_site-5]
            seq = temp2[2:index_site-10]
            Length = len(seq) 
           
    elif num == 4: #This is to get the Quality of the sequence read
        num = 0
        if index_site == -1:
            y=3
        
        else:
            
            count1+=1
            if Length > 19:
                count2+=1
                QualityT = line.rstrip()
                Quality = QualityT[2:Length+2]
                name1 = name1+umi+indx+str(Length)
                if indx == "TCTAG":
                    output1.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                      
                elif indx == "GCATA":
                    output2.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                        
                elif indx == "CTAGA":
                    output3.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                        
                elif indx == "AGCTA":
                    output4.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                elif indx == "ATCGT":
                    output5.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                        
                elif indx == "CGTAA":
                    output6.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                        
                elif indx == "GATCA":
                    output7.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n') 
                elif indx == "TAGAC":
                    output8.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                        
                else:
                    output9.write(name1+'\n'+ seq+'\n'+'+'+'\n'+Quality+'\n')
                            
file1.close()
output1.close()
output2.close()
output3.close()
output4.close()
output5.close()
output6.close()
output7.close()
output8.close()
output9.close()
print count/4
print count1
print count2



#Takes txt Solexa Sequence results and outputs them in fastq for
# Took the first 26bp, removed the 3'AAAAs

323682279
312205541
68154157


In [44]:

file1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RNA_RPF_Pool_S1_L002_R1_001.fastq' , 'r')
output1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF_TTAGGC.fastq' , 'w')
output2 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RNA_CGATGT.fastq' , 'w')
output3 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF_TGATGT.fastq' , 'w')
output4 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/I1_unaligned.fastq' , 'w')


def find_str(s,char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index
                    break
            index +=1
        
    return -1

count = 0
count1 = 0
count2 = 0
num = 0
read = 0
sequence = "AGTCAC"
exit = 0

for line in file1 :
    exit+=1
    
    count +=1
    num+=1
    if num == 1:
        name1 =line.rstrip()
            
    elif num ==2:  #The file has sequences on the 2nd line, every 4 lines
        temp2 = str(line.rstrip())
        Length = len(temp2)           
        index_site = (find_str(temp2, sequence))
                
        if index_site == -1:
            y=3
        else:
            indx = (temp2[index_site+6:index_site+8])
                       
    elif num == 4: #This is to get the Quality of the sequence read
        num = 0
        if index_site == -1:
            y=3
        
        #elif index_site < 40:
            #QualityT = line.rstrip()
            #name1 = name1+indx+str(index_site)
            #output5.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+QualityT+'\n')
            #count2+=1
        else:
            
            count1+=1
            
            QualityT = line.rstrip()
            name1 = name1+indx+str(index_site)
            if indx == "TT":
                output1.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+QualityT+'\n')
                      
            elif indx == "CG":
                output2.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+QualityT+'\n')
            
            elif indx == "TG":
                output3.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+QualityT+'\n')    
                
            else:
                output4.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+QualityT+'\n')
                count2+=1
                            
file1.close()
output1.close()
output2.close()
output3.close()

print count/4
print count1
print count2


386942584
374196097
12065202


In [4]:
file1 = open('D:/ZidLab/SequenceReads/eIF5aD/SRR5008135.fastq','r')
x=0

y = file1.readline()
print (y)
while x<100:
    x+=1
    print (file1.readline())
    

@SRR5008135.1.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1220:1969 length=50

TTGCTTCTTTGAAGCCAGAATTTGTTGATACTGTAGGCACCATCAATAGA

+SRR5008135.1.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1220:1969 length=50

??????????????????????????????????????????????????

@SRR5008135.2.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1333:1960 length=50

TCCCCGCTAGACGTCGTGGGCGTTTCCACTGTAGGCACCATCAATAGATC

+SRR5008135.2.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1333:1960 length=50

??????????????????????????????????????????????????

@SRR5008135.3.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1483:1973 length=50

GTGCCCGAGTTGTAATTTCTGTAGGCACCATCAATAGATCGGAAGAGCAC

+SRR5008135.3.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1483:1973 length=50

??????????????????????????????????????????????????

@SRR5008135.4.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1294:1975 length=50

TTTGCTCGAATATATTAGCATGGAATAATAGAATCTGTAGGCACCATCAA

+SRR5008135.4.1 160829_CIDR3_0241_AC9VM7ANXX:3:1101:1294:1975 length=50

??????????????????????????????????????????????????



In [56]:
file1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF-G_15m+Cyclo_ATCGT.fastq' , 'r')

output1 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF-G_15m+Cycloshort_ATCGT.fastq' , 'w')
output2 = open('D:/ZidLab/SequenceReads/Anna_9_17_18/RPF-G_15m+Cyclolong_ATCGT.fastq' , 'w')


count = 0
count1 = 0
count2 = 0
count3 =0
count17=0
count18=0
count19=0
count20=0
count21=0
count22=0
count23=0
count24=0
count25=0
count26=0
count27=0
count28=0
count29=0
count30=0
count31=0
count32=0

for line in file1 :
    exit+=1
    #if exit == 10000:
       #break
    count +=1
    num+=1
    if num == 1:
        name1 =line.rstrip()
            
    elif num ==2:  #The file has sequences on the 2nd line, every 4 lines
        temp2 = str(line.rstrip())
                   
        Length = len(temp2)
        if Length <18:
            count17+=1
        elif Length == 18:
            count18+=1
        elif Length == 19:
            count19+=1
        elif Length == 20:
            count20+=1    
        elif Length == 21:
            count21+=1    
        elif Length == 22:
            count22+=1    
        elif Length == 23:
            count23+=1      
        elif Length == 24:
            count24+=1    
        elif Length == 25:
            count25+=1      
        elif Length == 26:
            count26+=1    
        elif Length == 27:
            count27+=1      
        elif Length == 28:
            count28+=1    
        elif Length == 29:
            count29+=1      
        elif Length == 30:
            count30+=1    
        elif Length == 31:
            count31+=1      
        elif Length > 31:
            count32+=1       
            
            
            
            
    elif num == 4: #This is to get the Quality of the sequence read
        num = 0
        count1+=1
        if Length > 18 and Length < 23:
            count2+=1
            output1.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+line.rstrip()+'\n')
        elif Length >25 and Length < 31:
            output2.write(name1+'\n'+ temp2+'\n'+'+'+'\n'+line.rstrip()+'\n')
            count3+=1
                      
print count
print count1
print count2
print count3

print count17
print count18
print count19
print count20
print count21
print count22
print count23
print count24
print count25
print count26
print count27
print count28
print count29
print count30
print count31
print count32

file1.close()
output1.close()
output2.close()

77920380
19480095
8298627
3870425
0
0
0
3001627
2193086
3103914
2071209
2116940
2218366
1631237
2230666
8258
198
66
96
904432
